
Temperature  T is in joules/kb



In [1]:
from numba import jit
from PIL import Image
# from __future__ import division
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import random
%matplotlib inline


In [2]:
def initial_state(n):    
    state = np.random.choice([1, -1], size=(n, n))
    return state

In [3]:
def initialize_p(ln,beta,J=1):
  #one good things is to initialise this with ln = 6 , the maximum number of neighbours possible from all lattices. This is 
  p = np.ones(ln+1)
  for i in range(1,ln+1):
    p[i] = np.exp(-2*i*J*beta)
  return p

In [19]:
def step_update(state,p,J,H,acceptance,N = 10):  
#     for i in range(N**2):
    a = np.random.randint(0, N)
    b = np.random.randint(0, N)
    s =  state[a, b]
    nb = state[(a+1)%N,b] + state[a,(b+1)%N] + state[(a-1)%N,b] + state[a,(b-1)%N]
    cost = 2*s*(J*nb + H)
    # print(nb,cost,np.exp(-cost*beta),rand())
    if cost < 0:
      state[a, b] = -state[a, b]
      acceptance+=1
    elif rand() < p[np.abs(nb)]:
      state[a, b] = -state[a, b]
      acceptance+=1
    return state,acceptance

def step_update_with_energy(state,beta,J,H,energy,N = 50):
    a = np.random.randint(0, N)
    b = np.random.randint(0, N)
    s =  state[a, b]
    nb = state[(a+1)%N,b] + state[a,(b+1)%N] + state[(a-1)%N,b] + state[a,(b-1)%N]
    cost = 2*s*(J*nb + H)
    if cost < 0 or rand() < np.exp(-cost*beta):
        state[a, b] = -state[a, b]
        energy+=cost
    
    return state,energy


def step_update_with_energy_mag(state,p,J,H,energy,mag,acceptance,N=10):
    for i in range(N**2):
      a = np.random.randint(0, N)
      b = np.random.randint(0, N)
      s =  state[a, b]
      nb = state[(a+1)%N,b] + state[a,(b+1)%N] + state[(a-1)%N,b] + state[a,(b-1)%N]
      cost = 2*s*(J*nb + H)
      if cost < 0:
        if state[a,b] == 1:
            mag-=2
        else:
          mag+=2
        state[a, b] = -state[a, b]
        acceptance+=1
        energy+=cost
      elif rand() < p[np.abs(nb)]:
        if state[a,b] == 1:
            mag-=2
        else:
          mag+=2
        state[a, b] = -state[a, b]
        acceptance+=1
        energy+=cost
    return state,energy,mag,acceptance
        
    

In [5]:
def calcEnergy(state,J,H):
    energy = 0
    for i in range(len(state)):
        for j in range(len(state)):
            S = state[i,j]
            nb = state[(i+1)%N, j] + state[i,(j+1)%N] + state[(i-1)%N, j] + state[i,(j-1)%N]
            energy += -nb*S
    return J*(energy/2.) + H*(np.sum(state))


def calcMag(state):
    return np.sum(state)



## wolff

In [5]:
random.choice([(1,0),(0,1)])

(0, 1)

In [13]:
def ngbr(i,j,N):
  return [((i+1)%N,j),((i-1)%N,j),(i,(j+1)%N),(i,(j-1)%N)]
T = 2.5
beta = 1/T

steps = 10000
# state = intial_state(N)
def wolff_step(state,beta,p,N):
    a = np.random.randint(0, N)
    b = np.random.randint(0, N)
    
    s =  state[a, b]
    Pocket, Cluster = [(a,b)], [(a,b)]
    while Pocket != []:
        # print(Pocket)
        j = random.choice(Pocket)
        nbr = ngbr(j[0],j[1],N)
        for l in nbr:
            if state[l[0],l[1]] == state[j[0],j[1]] and l not in Cluster and rand() < p:
                Pocket.append(l)
                Cluster.append(l)
        Pocket.remove(j)
    print(Cluster)
    for j in Cluster:
        state[j[0],j[1]] *= -1
    return state


In [6]:
% matplotlib qt
N =10
temp = 1
J = 1
H = 0
state = initial_state(N)
beta = 1/temp
p  = 1.0 - np.exp(-2*beta) #for wolff
steps = int(1e3)
f = plt.figure(figsize=(10, 10), dpi=80);
plt.pause(20)
#wolff
for i in range(steps):
    plt.clf()
    X, Y = np.meshgrid(range(N), range(N))
    plt.pcolormesh(X, Y, state);
    plt.title('Time= {}, Metropolis Hastings'.format(i+1))
    plt.colorbar()
    state = wolff_step(state, 1.0/temp,p,N)
    plt.pause(0.0000003)

In [24]:
% matplotlib qt
N =50
temp = 1
J = 1
H = 0
state = initial_state(N)
beta = 1/temp
#metropolis   
steps = int(1e7)
p = initialize_p(4,beta,J) #for metropolis
f = plt.figure(figsize=(10, 10), dpi=80);
plt.pause(20)
k = np.arange(25,dtype = int)
ks = 2**k
idx=0
acceptance = 0
for i in range(steps):
#     print(i)
    if i == ks[idx]:
        plt.clf()
        X, Y = np.meshgrid(range(N), range(N))
        plt.pcolormesh(X, Y, state);
        plt.title('Time= {}, Metropolis Hastings'.format(i+1))
        plt.colorbar()
        idx+=1
    state,acceptance = step_update(state,p,J,H,acceptance,N)
    plt.pause(0.0003)


KeyboardInterrupt: 